<h4 align="center">BLAST Query Optimization using<br> Recursive Similarity Search for<br> SARS-CoV-2 Phylogenetic Tree Construction</h4>

--------

This notebook file contains programs using BioPython library which consist of two parts, the functions itself that will be used across the reasearch and the example usage of its functions. 

In [2]:
# library utama yang digunakan pada notebook

from Bio import AlignIO
from Bio import SeqIO
from Bio.Align import MultipleSeqAlignment
from Bio import Seq
import os
import re

#### 1. FASTA Header Extractor
This code block below were enlisted functions that was used in sequences filter & extraction as a part of lambda function

In [3]:
"""
fungsi utilitas
biasa digunakan pada lambda function
atau untuk tujuan lain
"""

def extract_gene_name(seq):
    description = seq.description
    x = re.findall("\[gene=(\w+)\]", description)
    return x[0]

def extract_protein_name(seq):
    description = seq.description
    x = re.findall("\[protein=([a-zA-Z\s0-9]+)\]", description)
    return x[0]

def get_modifier(seq, modifier):
    description = seq.description
    x = re.findall("\[" + modifier + "=([a-zA-Z\s0-9]+)\]", description)
    return x[0]

#### 2. FASTA Input Processing
This code block below were enlisted functions that was used to read, parse, filter, extract, and cut sequences

In [13]:
"""
membaca sequence dari filepath
menjadi list of sequence record
"""
def read_fasta(sequence_path):
    records = SeqIO.parse(sequence_path, 'fasta')
    return list(records)

""" 
fungsi ini menyatukan dua file sequences
menjadi satu dan menjaga terjadi duplikasinya
record berdasarkan sequenceID dengan returnnya
adalah seqIO records, perlu diingat pada NCBI
id record pada FASTA header memiliki format "lcl | ID_RECORD"
 """
def merge_sequences_list(sequences_list):
    merged_records = list()
    for sequences in sequences_list:
        for seq_record in sequences:
            exists = list(filter(lambda x: x.id == seq_record.id, merged_records))
            if (len(exists) == 0):
                merged_records.append(seq_record)
    return merged_records

""" 
fungsi ini membaca multiple file sequences
untuk difilter berdasarkan daftar id sequence
yang ingin tetap di-keep, di luar id tersebut akan
di takeout dengan returnnya adalah seqIO records
 """
def filter_sequences_list_by_ids(sequences_list, sequence_ids):
    merged_records = list()
    for sequences in sequences_list:
        for seq_record in sequences:
            match_ids = filter(lambda x: x.id == seq_record.id, sequence_ids)
            if (len(match_ids) > 0):
                merged_exists = filter(filtered_records, lambda x: x.id == "lcl " + seq_record.id)
                if (len(merged_exists) == 0):
                    merged_records.append(seq_record)
    return merged_records


""" 
fungsi ini membaca satu sequence untuk kemudian difilter
berdasarkan attribut protein pada fasta header
 """
def filter_sequences_by_protein(sequences, protein_name):
    filtered_records = list(filter(lambda x: (extract_protein_name(x) == protein_name), sequences))
    return filtered_records

"""
fungsi ini memotong setiap sequence dalam satu list seq records
berdasarkan maksimum length yang didefiniskan sebagai parameter
"""
def cut_sequences_length(sequences, max_length):
    cropped_sequences = map(lambda seq_record: (seq_record[:1000]), sequences)
    cropped_sequences = list(cropped_sequences)
    return cropped_sequences

"""
fungsi ini akan menulis objek list record SeqIO untuk
dijadikan menjadi file fasta pada direktori yang diinginkan
"""
def write_sequence_into_fasta(sequence, pathname):
    with open(pathname, 'w') as f:
        SeqIO.write(sequence, f, 'fasta')


#### Usage Example
below is an example code block that was used as a part of
the research method

In [6]:
# Search result 1
# read input and search result fasta (the CDS fasta format)
seq_search = read_fasta('./02-blast-search/search_result_1-cds.fasta')
seq_input = read_fasta('./01-inputs/NC_045512.2-cds.fasta')

# combine the input and search result fasta
seq_merged = merge_sequences_list([seq_search, seq_input])

# filter out by its protein code
seq_orf1ab = filter_sequences_by_protein(seq_merged, "ORF1ab polyprotein")

# cut first 1000 record
seq_orf1ab_1000 = cut_sequences_length(seq_orf1ab, 1000)

write_sequence_into_fasta(seq_orf1ab_1000, "./example_working_directory/search_result_1_orf1ab_1000.fasta")

In [7]:
# Search result 2
# read input and search result fasta (the CDS fasta format)
seq_search = read_fasta('./02-blast-search/search_result_2-cds.fasta')
seq_input = read_fasta('./01-inputs/OM065360.1-cds.fasta')

# combine the input and search result fasta
seq_merged = merge_sequences_list([seq_search, seq_input])

# filter out by its protein code
seq_orf1ab = filter_sequences_by_protein(seq_merged, "ORF1ab polyprotein")

# cut first 1000 record
seq_orf1ab_1000 = cut_sequences_length(seq_orf1ab, 1000)

write_sequence_into_fasta(seq_orf1ab_1000, "./example_working_directory/search_result_2_orf1ab_1000.fasta")

In [14]:
# Search result 3
# read input and search result fasta (the CDS fasta format)
seq_search = read_fasta('./02-blast-search/search_result_3-cds.fasta')
seq_input = read_fasta('./01-inputs/OR184938.1_cds.fasta')

# combine the input and search result fasta
seq_merged = merge_sequences_list([seq_search, seq_input])

# filter out by its protein code
seq_orf1ab = filter_sequences_by_protein(seq_merged, "ORF1ab polyprotein")

# cut first 1000 record
seq_orf1ab_1000 = cut_sequences_length(seq_orf1ab, 1000)

write_sequence_into_fasta(seq_orf1ab_1000, "./example_working_directory/search_result_3_orf1ab_1000.fasta")

#### Aligning and tree construction
The sequence then can be aligned using muscle program and the tree
can be generated using biopython
<br>
[see the implementation on this notebook](./main_phylo.ipynb)